### Imports

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

import uuid
import json
import boto3
import constants
import utilities
from datetime import datetime
from dynamoDBHelper import DynamoDBHelper
from userJobHelper import UserJobHelper
from s3Helper import S3Helper

###  SESHelper Class Usage & Testing

In [1]:
import json
import boto3
from datetime import datetime

import constants
import utilities
from userJobHelper import UserJobHelper
from sesHelper import SesHelper
from sesHelper import SesEmailMessage
from userJobHelper import UserJobHelper

userEmailId = 'pasamoha@amazon.com'
jobId = 'baf497bb-c902-42a1-ba06-fcad41e264e'

region = boto3.Session().region_name
source = 'aws-strategic-ugh-pca@amazon.com'
replyToAddress = source
messageToAddresses = userEmailId
messageCcAddresses = userEmailId
messageBccAddresses = userEmailId
presentationFilename = 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.pptx'

messageSubject = 'Download the File' 
messageBody = 'Download the File Here...'
emailMessage = SesEmailMessage(source, SesEmailMessage.createDestination(messageToAddresses, messageCcAddresses, messageBccAddresses),
                               SesEmailMessage.createMessage(messageSubject, messageBody), replyToAddress)
#emailMessage = SesEmailMessage('aws-strategic-ugh-pca@amazon.com', SesEmailMessage.createDestination(messageToAddresses, messageCcAddresses, messageBccAddresses),
#                               SesEmailMessage.createMessage(messageSubject, messageBody), replyToAddress)
print(emailMessage)

#Create an instance of the SQS helper class
sesHelper = SesHelper(region)
#sesHelper.sendEmail(emailMessage)

sesHelper.sendCustomVerificationEmail(userEmailId, '')

Apr 13 2022 17:51:37 - SendCustomVerificationEmail -> request: emailAddress=pasamoha@amazon.com
 templateName=
 configurationSetName=None



###  S3Helper Class Usage & Testing

In [1]:
import os
import boto3
import constants

from sagemaker.session import Session
from s3Helper import S3Helper
from utilities import FileHelper

region = boto3.Session().region_name

bucketname = 'unicorn-gym-hackathon-presentation-creator-app'
jobPrefix = f'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/'
slidesPrefix = jobPrefix + 'slides/'
videoFilename = jobPrefix + 'my-presentation.mp4'
presentationFileName = jobPrefix + 'my-presentation.pptx'
presentationTemplateFilename = jobPrefix + 'presentationtemplate.pptx'
slideTranscriptionsFilename = jobPrefix + 'slidetranscriptions.json'
transcriptionsFilename = jobPrefix + 'transcriptions.json'
slideTransitionsFilename = jobPrefix + 'slidetransitions.json'

#current directory
localDownloadPath = os.getcwd() + constants.CHAR__FORWARD_SLASH
print(localDownloadPath)
localUploadPath = localDownloadPath  + 'uploads/'
print(localUploadPath)
localSlideTransitionsFilename = os.path.basename(slideTransitionsFilename)
print(localSlideTransitionsFilename)
localTranscriptionsFilename = os.path.basename(transcriptionsFilename)
print(localTranscriptionsFilename)
print(f'transcriptionsFilename={transcriptionsFilename}, {localUploadPath + localTranscriptionsFilename}')

print(f'FullPath={S3Helper.getFullPath(bucketname, videoFilename)}')

#Create an instance of the S3 helper class
s3Helper = S3Helper(region)
prefix = 'data/users/pasamoha@amazon.com'
notPresentPrefix = 'data/users/test@amazon.com'
print(f'prefixExists={s3Helper.prefixExists(bucketname, prefix)}')
print(f'prefixExists={s3Helper.prefixExists(bucketname, prefix + "/")}')
print(f'prefixExists={s3Helper.prefixExists(bucketname, notPresentPrefix)}')
print(f'prefixExists={s3Helper.prefixExists(bucketname, notPresentPrefix + "/")}')

/root/unicorn-hackathon/presentation-creator-app/
/root/unicorn-hackathon/presentation-creator-app/uploads/
slidetransitions.json
transcriptions.json
transcriptionsFilename=data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/transcriptions.json, /root/unicorn-hackathon/presentation-creator-app/uploads/transcriptions.json
FullPath=s3://unicorn-gym-hackathon-presentation-creator-app/data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4
Apr 13 2022 17:01:41 - PrefixExists -> request = bucketName=unicorn-gym-hackathon-presentation-creator-app
 prefix=data/users/pasamoha@amazon.com

Apr 13 2022 17:01:41 - GetObjects -> request = bucketName=unicorn-gym-hackathon-presentation-creator-app
 prefix=data/users/pasamoha@amazon.com
 maxItems=1
 pageSize=1

prefixExists=True
Apr 13 2022 17:01:41 - PrefixExists -> request = bucketName=unicorn-gym-hackathon-presentation-creator-app
 prefix=data/users/pasamoha@amazon.com/

Apr 13 2022 17:01:41 - GetObject

In [ ]:
#Test Upload/Download transcriptionsFilename
s3Helper.uploadFile(bucketname, transcriptionsFilename, localUploadPath + localTranscriptionsFilename)
s3Helper.downloadFile(bucketname, transcriptionsFilename, localDownloadPath + localTranscriptionsFilename)

In [ ]:
#Test Download Pre-signed URL
transcriptionsFilePresignedUrl = s3Helper.createDownloadPresignedUrl(bucketname, transcriptionsFilename)
print(transcriptionsFilePresignedUrl)

#Test Download Pre-signed URL
transcriptionsFileDownloadPresignedUrl = s3Helper.createDownloadPresignedUrl(bucketname, transcriptionsFilename)
print(transcriptionsFileDownloadPresignedUrl)

In [ ]:
#Test Upload Pre-signed URL
presentationVideoFile='data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4'
presentationVideoFileUploadPresignedUrl = s3Helper.createUploadPresignedUrl(bucketname, presentationVideoFile)
print(presentationVideoFileUploadPresignedUrl)

### SQSHelper Class Usage & Testing

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

import uuid
import json
import boto3
import constants
import utilities

from datetime import datetime
from s3Helper import S3Helper
from sqsHelper import SqsHelper
from sqsHelper import SqsMessage
from utilities import AttributeHelper
from utilities import AttributeDataTypes
from userJobHelper import UserJobHelper

region = boto3.Session().region_name
queueName = 'ugh-pca_Send_Email'
userEmailId = 'pasamoha@amazon.com'
jobId = 'baf497bb-c902-42a1-ba06-fcad41e264e'
presentationFilename = 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.pptx'

#Create an instance of the SQS helper class
sqsHelper = SqsHelper(region, queueName)
messageBody = constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_PRESENTATION
#messageAttributes =dict({
#                constants.UGH_PC_DDB_PK__USER_EMAIL_ID:userEmailId,
#                constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:presentationFilename
#            })

messageAttributes = {}

attributes = {
                constants.UGH_PC_DDB_PK__USER_EMAIL_ID: { AttributeHelper.ATTRIBUTE_FLD__DATATYPE: AttributeDataTypes.StringList, AttributeHelper.ATTRIBUTE_FLD__VALUE : {userEmailId, 'shubhol@amazon.com'}},
                constants.UGH_PC_DDB_SK__USER_JOB_ID:jobId,
                constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:presentationFilename
            }
print(AttributeHelper.createAttributes(attributes))

attributes = {
                constants.UGH_PC_DDB_PK__USER_EMAIL_ID:userEmailId,
                constants.UGH_PC_DDB_SK__USER_JOB_ID:jobId,
                constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:presentationFilename
            }

transformedAttributes = AttributeHelper.createAttributes(attributes)
print(transformedAttributes)
print(f'reTransformedAttributes={AttributeHelper.getAttributes(transformedAttributes)}')

sqsEvent = {
  "Records": [
    {
      "messageId": "059f36b4-87a3-44ab-83d2-661975830a7d",
      "receiptHandle": "AQEBwJnKyrHigUMZj6rYigCgxlaS3SLy0a...",
      "body": "Send Email-1",
      "messageAttributes": {'user_email_id': {'DataType': 'String', 'StringValue': 'pasamoha@amazon.com'}, 'job_id': {'DataType': 'String', 'StringValue': 'baf497bb-c902-42a1-ba06-fcad41e264e'}, 'PresentationFileName': {'DataType': 'String', 'StringValue': 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.pptx'}},
      "md5OfBody": "098f6bcd4621d373cade4e832627b4f6",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-west-2:192399867703:ugh-pca_Send_Email",
      "awsRegion": "us-west-2"
    },
    {
      "messageId": "446c6e66-983e-4728-a80e-de2799ff4119",
      "receiptHandle": "AQEBwJnKyrHigUMZj6rYigCgxlaS3SLy0a...",
      "body": "Send Email-2",
      "messageAttributes": {'user_email_id': {'DataType': 'String', 'StringValue': 'shubhol@amazon.com'}, 'job_id': {'DataType': 'String', 'StringValue': 'f105cce5-ca71-4e87-9689-0f2683a635ea'}, 'PresentationFileName': {'DataType': 'String', 'StringValue': 'data/users/shubhol@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.pptx'}},
      "md5OfBody": "098f6bcd4621d373cade4e832627b4f6",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-west-2:192399867703:ugh-pca_Send_Email",
      "awsRegion": "us-west-2"
    }
  ]
}

sqsMessages = sqsHelper.parseMessageFromEvent(sqsEvent)
for message in sqsMessages:
    print(f'Messagebody={message.messageBody} MessageAttributes={message.messageAttributes}')

In [ ]:
sqsHelper.sendMessage(queueName, SqsMessage(messageBody, attributes))

### SNSHelper Class Usage & Testing

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

import uuid
import json
import boto3
from datetime import datetime

import constants
import utilities
from utilities import Helper
from utilities import Logger
from utilities import LogLevel
from s3Helper import S3Helper
from snsHelper import SnsHelper
from snsHelper import SnsS3Message
from sqsHelper import SqsHelper
from sqsHelper import SqsMessage
from utilities import FileHelper
from utilities import FileNameOption
from userJobHelper import UserJobHelper

In [ ]:
snsEvent = {
  'Records': [
    {
      'EventSource': 'aws:sns',
      'EventVersion': '1.0',
      'EventSubscriptionArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files:bd5e94a0-f7b9-4d49-87f6-0c804e631466',
      'Sns': {
        'Type': 'Notification',
        'MessageId': '5b91bc36-6e6d-5f90-89f3-e6ff2d1fc4f5',
        'TopicArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files',
        'Subject': 'Amazon S3 Notification',
        'Message': {
          'Records': [
            {
              'eventVersion': '2.1',
              'eventSource': 'aws:s3',
              'awsRegion': 'us-west-2',
              'eventTime': '2022-04-07T18:39:02.075Z',
              'eventName': 'ObjectCreated:CompleteMultipartUpload',
              'userIdentity': {
                'principalId': 'AWS:AROASZS7M743Y5CU5QFWF:pasamoha-Isengard'
              },
              'requestParameters': {
                'sourceIPAddress': '76.102.132.19'
              },
              'responseElements': {
                'x-amz-request-id': 'SH9WWBTWD2BRMWQ3',
                'x-amz-id-2': 'C5FKxuPCvLWNixhjyABy+B6nz65eG+5R87WAYYt7pQz/NDXrCjdRzXNYCYQCIfe7TcDVIn4JDmYVyGgPUvaMSJrFyPllDvpO'
              },
              's3': {
                's3SchemaVersion': '1.0',
                'configurationId': 'ugh-pca_S3_Notify_SNS',
                'bucket': {
                  'name': 'unicorn-gym-hackathon-presentation-creator-app',
                  'ownerIdentity': {
                    'principalId': 'A2HS2VBLSNTHRD'
                  },
                  'arn': 'arn:aws:s3:::unicorn-gym-hackathon-presentation-creator-app'
                },
                'object': {
                  'key': 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4',
                  'size': 36688467,
                  'eTag': '25ab102bf2929ad2d1f8a4e5e4d6af4f-3',
                  'sequencer': '00624F2FBCC07B3D66'
                }
              }
            },
                {
                  'eventVersion': '2.1',
                  'eventSource': 'aws:s3',
                  'awsRegion': 'us-west-2',
                  'eventTime': '2022-04-07T18:39:02.075Z',
                  'eventName': 'ObjectCreated:CompleteMultipartUpload',
                  'userIdentity': {
                    'principalId': 'AWS:AROASZS7M743Y5CU5QFWF:pasamoha-Isengard'
                  },
                  'requestParameters': {
                    'sourceIPAddress': '76.102.132.19'
                  },
                  'responseElements': {
                    'x-amz-request-id': 'SH9WWBTWD2BRMWQ3',
                    'x-amz-id-2': 'C5FKxuPCvLWNixhjyABy+B6nz65eG+5R87WAYYt7pQz/NDXrCjdRzXNYCYQCIfe7TcDVIn4JDmYVyGgPUvaMSJrFyPllDvpO'
                  },
                  's3': {
                    's3SchemaVersion': '1.0',
                    'configurationId': 'ugh-pca_S3_Notify_SNS',
                    'bucket': {
                      'name': 'unicorn-gym-hackathon-presentation-creator-app',
                      'ownerIdentity': {
                        'principalId': 'A2HS2VBLSNTHRD'
                      },
                      'arn': 'arn:aws:s3:::unicorn-gym-hackathon-presentation-creator-app'
                    },
                    'object': {
                      'key': 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4',
                      'size': 36688467,
                      'eTag': '25ab102bf2929ad2d1f8a4e5e4d6af4f-3',
                      'sequencer': '00624F2FBCC07B3D66'
                    }
                  }
                }            
          ]
        }
      }
    }
  ]
}

In [5]:
snsEvent = {
  'Records': [
    {
      'EventSource': 'aws:sns',
      'EventVersion': '1.0',
      'EventSubscriptionArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files:bd5e94a0-f7b9-4d49-87f6-0c804e631466',
      'Sns': {
        'Type': 'Notification',
        'MessageId': '5b91bc36-6e6d-5f90-89f3-e6ff2d1fc4f5',
        'TopicArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files',
        'Subject': 'Amazon S3 Notification',
        'Message': {
          'Records': [
            {
              'eventVersion': '2.1',
              'eventSource': 'aws:s3',
              'awsRegion': 'us-west-2',
              'eventTime': '2022-04-07T18:39:02.075Z',
              'eventName': 'ObjectCreated:CompleteMultipartUpload',
              'userIdentity': {
                'principalId': 'AWS:AROASZS7M743Y5CU5QFWF:pasamoha-Isengard'
              },
              'requestParameters': {
                'sourceIPAddress': '76.102.132.19'
              },
              'responseElements': {
                'x-amz-request-id': 'SH9WWBTWD2BRMWQ3',
                'x-amz-id-2': 'C5FKxuPCvLWNixhjyABy+B6nz65eG+5R87WAYYt7pQz/NDXrCjdRzXNYCYQCIfe7TcDVIn4JDmYVyGgPUvaMSJrFyPllDvpO'
              },
              's3': {
                's3SchemaVersion': '1.0',
                'configurationId': 'ugh-pca_S3_Notify_SNS',
                'bucket': {
                  'name': 'unicorn-gym-hackathon-presentation-creator-app',
                  'ownerIdentity': {
                    'principalId': 'A2HS2VBLSNTHRD'
                  },
                  'arn': 'arn:aws:s3:::unicorn-gym-hackathon-presentation-creator-app'
                },
                'object': {
                  'key': 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4',
                  'size': 36688467,
                  'eTag': '25ab102bf2929ad2d1f8a4e5e4d6af4f-3',
                  'sequencer': '00624F2FBCC07B3D66'
                }
              }
            }            
          ]
        }
      }
    }
  ]
}

###  TranscribeHelper Class Usage & Testing

In [6]:
import sagemaker 
from sagemaker import get_execution_role

import boto3

import constants
import utilities
from utilities import Logger
from utilities import LogLevel
from datetime import datetime
from s3Helper import S3Helper
from userJobHelper import UserJobHelper
from transcribeHelper import TranscribeHelper
from transcribeHelper import TranscribeJobParameters


def processEvent(snsEvent) -> bool:
    processedEvent = False
    jobKeys = UserJobHelper.processVideoFileUploadEvent(snsEvent)
    for jobKey in jobKeys:          
        userEmailId = jobKey[constants.UGH_PC_DDB_PK__USER_EMAIL_ID]
        jobId = jobKey[constants.UGH_PC_DDB_SK__USER_JOB_ID]
        processedEvent = processAudio(userEmailId, jobId)
    return processedEvent


def processAudio(userEmailId: str, jobId: str)-> bool:
    Logger.log(f'ProcessAudio -> request: userEmailId={userEmailId}{constants.CHAR__NEWLINE} jobId={jobId}')
    
    try:
        userJobHelper = UserJobHelper(userEmailId, jobId)
        Logger.log(f'ProcessAudio -> Deleting: transcriptionsFilename={userJobHelper.transcriptionsFilenameResolved}')
        s3Helper = S3Helper(boto3.Session().region_name)
        s3Helper.deleteFile(userJobHelper.bucketname, userJobHelper.transcriptionsFilenameResolved) 
        transcribeJobParameters = TranscribeJobParameters(S3Helper.getFullPath(userJobHelper.bucketname, userJobHelper.videoFilename), userJobHelper.bucketname, userJobHelper.transcriptionsFilenameResolved)
        transcribeHelper = TranscribeHelper()
        transcribeJob = transcribeHelper.startTranscribeJob(transcribeJobParameters)
        Logger.log(f'ProcessAudio -> response: transcribeJob={transcribeJob}')
    except Exception as e:
        Logger.log(f'ProcessAudio -> ERROR = {e}', LogLevel.Exception)    
        return False
    return True

In [7]:
processEvent(snsEvent)

Apr 13 2022 09:47:10 - ProcessVideoFileUploadEvent -> Event={'Records': [{'EventSource': 'aws:sns', 'EventVersion': '1.0', 'EventSubscriptionArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files:bd5e94a0-f7b9-4d49-87f6-0c804e631466', 'Sns': {'Type': 'Notification', 'MessageId': '5b91bc36-6e6d-5f90-89f3-e6ff2d1fc4f5', 'TopicArn': 'arn:aws:sns:us-west-2:192399867703:ugh-pca_Process_Job_Files', 'Subject': 'Amazon S3 Notification', 'Message': {'Records': [{'eventVersion': '2.1', 'eventSource': 'aws:s3', 'awsRegion': 'us-west-2', 'eventTime': '2022-04-07T18:39:02.075Z', 'eventName': 'ObjectCreated:CompleteMultipartUpload', 'userIdentity': {'principalId': 'AWS:AROASZS7M743Y5CU5QFWF:pasamoha-Isengard'}, 'requestParameters': {'sourceIPAddress': '76.102.132.19'}, 'responseElements': {'x-amz-request-id': 'SH9WWBTWD2BRMWQ3', 'x-amz-id-2': 'C5FKxuPCvLWNixhjyABy+B6nz65eG+5R87WAYYt7pQz/NDXrCjdRzXNYCYQCIfe7TcDVIn4JDmYVyGgPUvaMSJrFyPllDvpO'}, 's3': {'s3SchemaVersion': '1.0', 'configurati

True

In [ ]:
jobKeys = UserJobHelper.processVideoFileUploadEvent(snsEvent)
for jobKey in jobKeys:          
    userEmailId = jobKey[constants.UGH_PC_DDB_PK__USER_EMAIL_ID]
    jobId = jobKey[constants.UGH_PC_DDB_SK__USER_JOB_ID]
    print(f'userEmailId={userEmailId} JobId={jobId}')
    # Create New Job when Job ID exists
    userJobHelper = UserJobHelper(userEmailId, jobId)
    #Call process Video or Audio here
    #Helper.sleepRandomSeconds(5)
    userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_PROCESSING_AUDIO, constants.UGH_PC_DDB_FLDVALUE__STATUS_COMPLETE)

In [ ]:
snsHelper = SnsHelper(boto3.Session().region_name)
snsMessages = snsHelper.parseMessageFromS3Event(snsEvent)
for message in snsMessages:
    print(f'BucketName={message.bucketName} Key={message.key}')

### DynamoDBHelper Class Usage & Testing

In [ ]:
dynamoDb = DynamoDBHelper()

In [ ]:
from appConfigHelper import AppConfigHelper
appConfigHelper = AppConfigHelper()
print(appConfigHelper.bucketname)

In [ ]:
#Test DynamoDBHelper
jobId = str(uuid.uuid4())
currentdatetime = str(datetime.now())
userEmailId = 'pasamoha@amazon.com'

dynamoDb = DynamoDBHelper()
#Put Item
dynamoDb.putItem(constants.UGH_PC_DDB_TABLENAME__USER_JOBS, 
    item={
        constants.UGH_PC_DDB_PK__USER_EMAIL_ID: userEmailId,
        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId,
        constants.UGH_PC_DDB_FLD__LAST_UPDATED:currentdatetime,
        constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES:False,
        constants.UGH_PC_DDB_FLD__JOB_STATUS:"Job Started"
    }
)

In [ ]:
userEmailId = 'pasamoha@amazon.com'
jobId = '2c4da419-ad0a-4757-9e7a-d6940e134d93'
#Get Item
item = dynamoDb.getItem(constants.UGH_PC_DDB_TABLENAME__USER_JOBS, 
   itemKey={
        constants.UGH_PC_DDB_PK__USER_EMAIL_ID: userEmailId,
        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId      
    }
)
print (item)

In [ ]:
#Update Item
response = dynamoDb.updateItem(constants.UGH_PC_DDB_TABLENAME__USER_JOBS, 
   itemKey={
        constants.UGH_PC_DDB_PK__USER_EMAIL_ID: userEmailId,
        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId      
    },
    attributes={
        constants.UGH_PC_DDB_FLD__JOB_STATUS: "Presentation Created"
        #constants.UGH_PC_DDB_FLD__LAST_UPDATED: str(datetime.now())
    }
)
print (response)

In [ ]:
#Update Item
response = dynamoDb.updateItem(constants.UGH_PC_DDB_TABLENAME__USER_JOBS, 
   itemKey={
        constants.UGH_PC_DDB_PK__USER_EMAIL_ID: userEmailId,
        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId      
    },
    attributes={
        constants.UGH_PC_DDB_FLD__JOB_ARTIFACTS: {
            "VideoFilename":"my-presentation1.mp4",
            "PresentationFileName":"my-presentation1.pptx"
        }
        #constants.UGH_PC_DDB_FLD__LAST_UPDATED: str(datetime.now())
    }
)
print (response)

In [ ]:
#Delete Item
dynamoDb.deleteItem(constants.UGH_PC_DDB_TABLENAME__USER_JOBS, 
   itemKey={
        constants.UGH_PC_DDB_PK__USER_EMAIL_ID: userEmailId,
        constants.UGH_PC_DDB_SK__USER_JOB_ID: newJobId      
    })

###  FileHelper Class Usage & Testing

In [ ]:
from utilities import FileHelper
from utilities import FileNameOption

filename = '/root/dir/sub.exten/file.data.1part.2parts.txt'
print(f'Empty filename = {FileHelper.getFileName(None)}')
print(f"Filename = {FileHelper.getFileName(filename, FileNameOption.Filename)}")
print(f"FilenameOnly = {FileHelper.getFileName(filename, FileNameOption.FilenameOnly)}")
print(f"FilenameExtension = {FileHelper.getFileName(filename, FileNameOption.FilenameExtension)}")
print(f"FilenameLastExtension = {FileHelper.getFileName(filename, FileNameOption.FilenameLastExtension)}")
print(f"FilenamePath = {FileHelper.getFileName(filename, FileNameOption.FilenamePath)}")

#filename = '/root/dir/sub.exten/data/'
filename = 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4'
print(f"FilenamePath = {FileHelper.getFileName(filename, FileNameOption.FilenamePath)}")
print(f"FilenameDirectory = {FileHelper.getFileName(filename, FileNameOption.FilenameDirectory)}")

### UserJobHelper Class Usage & Testing

In [ ]:
userEmailId = 'pasamoha@amazon.com'
jobId = 'baf497bb-c902-42a1-ba06-fcad41e264e' #Enter an existing JobId

# Create New Job when Job ID exists
userJobHelper = UserJobHelper(userEmailId, jobId)

# Use the following properties from UserJobHelper class
print(userJobHelper.overwriteNotes)
print(userJobHelper.presentationTemplateFilename)
print(userJobHelper.videoFilename)
print(userJobHelper.presentationFilename)
print(userJobHelper.transcriptionsFilename) 
print(userJobHelper.slideTranscriptionsFilename) 
print(userJobHelper.slideTransitionsFilename) 
print(userJobHelper.slidePrefix) 
print(userJobHelper.bucketname) 
print(userJobHelper.jobStatus) 

In [ ]:
# Get Job Key From Filename
filename = 'data/users/pasamoha@amazon.com/baf497bb-c902-42a1-ba06-fcad41e264e/my-presentation.mp4'
jobKeys = UserJobHelper.getJobKeyFromFilename(filename)
print(jobKeys)

jobStatus = userJobHelper.getJobStatus()
print(jobStatus)

In [ ]:
# Get Existing Job
#Static method
job = UserJobHelper.getJob(userEmailId, jobId)

In [ ]:
# Create New Job
userJobHelper = UserJobHelper(userEmailId)
newJobId = userJobHelper.jobId

In [ ]:
# Get Existing Job
#Instance method
#newJob = userJobHelper.getJob()
#print(newJob)

In [ ]:
# Update Job Files
result = userJobHelper.updateJobInputFiles("my-presentation.mp4", "my-presentation.pptx")

In [ ]:
# Update Status
result = userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_FILES_UPLOAD_COMPLETE)

# Update Status with conditional current status
result = userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_FILES_UPLOAD_COMPLETE)

In [ ]:
def createJob(userEmailId: str, videoFilename: str, presentationFilename = constants.CHAR__EMPTY, overwriteNotes = False):
    # Create New Job
    userJobHelper = UserJobHelper(userEmailId, constants.CHAR__EMPTY, overwriteNotes)
    userJobHelper.updateJobInputFiles(videoFilename, presentationFilename)
    
    #Create an instance of the S3 helper class
    s3Helper = S3Helper(boto3.Session().region_name)
    videoUploadPresignedUrl = s3Helper.createUploadPresignedUrl(userJobHelper.bucketname, userJobHelper.videoFilename)
    if presentationFilename != None and presentationFilename != constants.CHAR__EMPTY:
        presentationUploadPresignedUrl = s3Helper.createUploadPresignedUrl(userJobHelper.bucketname, userJobHelper.presentationFilename)
    else:
        presentationUploadPresignedUrl = constants.CHAR__EMPTY
    
    return json.dumps({
        constants.UGH_PC_DDB_SK__USER_JOB_ID: userJobHelper.jobId,
        constants.UGH_PC_DDB_FLD__AC_VIDEO_FILENAME:  videoUploadPresignedUrl,
        constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:  presentationUploadPresignedUrl
    })

result = createJob('pasamoha@amazon.com', "new-presentation.mp4", "new-presentation.pptx", True)
print(result)

### Imports

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

import uuid
import json
import constants
import utilities
from datetime import datetime
from dynamoDBHelper import DynamoDBHelper
from userJobHelper import UserJobHelper
from presentationHelper import PresentationHelper

### PresentationHelper Class Usage and Testing

In [ ]:
# python-pptx is library used to create/edit a PowerPoint (.pptx) files. This won’t work on MS office 2003 and previous versions.  We can add shapes, paragraphs, texts and slides and much more thing using this library.
# Install the library using pip
pip install python-pptx

##### Functions to merge slide transitions and transcriptions to create slide transciptions file

In [ ]:
# Test merging the slidetransitions and transcriptions
slideTransistionFilename = "slidetransition.json"
transcriptionFilename = "transcription.json"

presentationHelper = PresentationHelper(slideTransistionFilename, transcriptionFilename)
PresentationHelper.toJsonFile(presentationHelper.getNotes(), 'slidetranscriptions.json')

##### Functions to create/merge slide transciptions  as notes in to the powerpoint presentation file

In [ ]:
print(PresentationHelper.getFormatedSlideNotes(False, "This is a slide number 12. End of the presentation.", "THIS WAS THE PEREVIOUS NOTE"))

##### Testing the above functions for 4 different use cases.

In [ ]:
# USECASE-1: Notes gathered is exactly equal to the total slides available in the PPT (Full PPT available) - APPEND TO EXISTING NOTES - Check Slide #6
PresentationHelper.addSlideNotesToPresentation({
            constants.UGH_PC_DDB_FLD__AC_SLIDE_TRANSCRIPTIONS_FILENAME: "slidetranscriptions.json",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME: "my-presentation.pptx",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_TEMPLATE_FILENAME: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__AC_SLIDES_PREFIX: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES : False
        })

In [ ]:
# USECASE-2: Notes gathered is exactly equal to the total slides available in the PPT (Full PPT available) - OVERWRITE EXISTING NOTES - Check Slide #6
PresentationHelper.addSlideNotesToPresentation({
            constants.UGH_PC_DDB_FLD__AC_SLIDE_TRANSCRIPTIONS_FILENAME: "slidetranscriptions.json",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME: "my-presentation.pptx",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_TEMPLATE_FILENAME: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__AC_SLIDES_PREFIX: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES : True
        })

In [ ]:
# USECASE-3: Notes gathered is less than total slides available in the PPT (Full PPT available)
PresentationHelper.addSlideNotesToPresentation({
            constants.UGH_PC_DDB_FLD__AC_SLIDE_TRANSCRIPTIONS_FILENAME: "slidetranscriptions-missing-notes.json",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME: "my-presentation.pptx",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_TEMPLATE_FILENAME: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__AC_SLIDES_PREFIX: constants.CHAR__EMPTY,
            constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES : False
        })

In [ ]:
import uuid
import shutil
import os
from utilities import Helper

# USECASE-4: PPT NOT available. Genrate PPT from the collected images.
presentationFilenametemplate = 'my-presentation-template.pptx'
presentationFilename = f'my-presentation-{Helper.createGuid()}.pptx'
shutil.copyfile(presentationFilenametemplate, presentationFilename)
PresentationHelper.addSlideNotesToPresentation({
            constants.UGH_PC_DDB_FLD__AC_SLIDE_TRANSCRIPTIONS_FILENAME: "slidetranscriptions.json",
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME: presentationFilename,
            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_TEMPLATE_FILENAME: presentationFilenametemplate,
            constants.UGH_PC_DDB_FLD__AC_SLIDES_PREFIX: "slides/",
            constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES : False
        })    

### End-to-End testing using S3 as file source/destination

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep

import boto3
import uuid
import json
import constants
import utilities
from datetime import datetime
from dynamoDBHelper import DynamoDBHelper
from userJobHelper import UserJobHelper
from s3Helper import S3Helper

# Create New Job - 
# 1 - TRIGGER WHEN USER UPLOADS FILES 

userInformationJson = {
    constants.UGH_PC_DDB_PK__USER_EMAIL_ID:"pasamoha@amazon.com",
    constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES: False
}
userJobHelper = UserJobHelper(userInformationJson[constants.UGH_PC_DDB_PK__USER_EMAIL_ID], constants.CHAR__EMPTY
userInformationJson[UGH_PC_DDB_FLD__OVERWRITE_NOTES])
userJobId = userJobHelper.jobId
#s3Helper = S3Helper(region)

# 2 - GET PRESIGNED URL FOR UPLOAD
userJobHelper = UserJobHelper(userEmailId)
newJobId = userJobHelper.jobId

userUploadPresignedUrl = s3Helper.createUploadPresignedUrl(bucketname, transcriptionsFilename)

# 3 - TRIGGER WHEN USER FILE UPLOADS IS COMPLETED 
sqsInputEventJson = {
    constants.UGH_PC_DDB_PK__USER_EMAIL_ID:"pasamoha@amazon.com"
    constants.UGH_PC_DDB_SK__USER_JOB_ID:"baf497bb-c902-42a1-ba06-fcad41e264e"
}

# 4 - Process Video File
# 4A - Create Slide Transitions
# 4B - Create Transcriptions

    
#5 - Merging the slidetransitions and transcriptions
#6 - Create the presentation and insert notes in to the slides

#7 - TRIGGER EMAIL


In [ ]:
import sagemaker 
from sagemaker import get_execution_role

import os
import boto3
import uuid
import json
from time import gmtime, strftime, sleep

import constants
import utilities
from utilities import Logger
from utilities import LogLevel
from datetime import datetime
from s3Helper import S3Helper
from sqsHelper import SqsHelper
from sqsHelper import SqsMessage
from utilities import FileHelper
from utilities import FileNameOption
from dynamoDBHelper import DynamoDBHelper
from userJobHelper import UserJobHelper
#from presentationHelper import PresentationHelper


def lambda_handler(event, context):
    result =  {
        'statuscode': 200,
        'body': 'Job Completed Successfully!!!'
    }
    if processEvent(event) == False:
        result =  {
            'statuscode': 500,
            'body': 'Job Failed!!!'
        }
    return result
    

def processEvent(sqsEvent) -> bool:
    processedEvent = False
    #Create an instance of the SQS helper class
    sqsHelper = SqsHelper(boto3.Session().region_name)
    sqsMessages = sqsHelper.parseMessageFromEvent(sqsEvent)
    
    for message in sqsMessages:
        Logger.log(f'ProcessEvent -> Messagebody={message.messageBody}{constants.CHAR__NEWLINE} MessageAttributes={message.messageAttributes}')
        if constants.UGH_PC_DDB_PK__USER_EMAIL_ID in message.messageAttributes:
            userEmailId = message.messageAttributes[constants.UGH_PC_DDB_PK__USER_EMAIL_ID]
            if constants.UGH_PC_DDB_SK__USER_JOB_ID in message.messageAttributes:
                jobId = message.messageAttributes[constants.UGH_PC_DDB_SK__USER_JOB_ID]
                Logger.log(f'ProcessEvent -> User={userEmailId}{constants.CHAR__NEWLINE} jobId={jobId}')
                processedEvent = createPresentation(userEmailId, jobId)
    return processedEvent


def createPresentation(userEmailId: str, jobId: str) -> bool:
    createdPresentation = False
    Logger.log(f'CreatePresentation -> User={userEmailId}{constants.CHAR__NEWLINE} jobId={jobId}')
    
    try:
        # Create New Job when Job ID exists
        userJobHelper = UserJobHelper(userEmailId, jobId)
        Logger.log(f'CreatePresentation -> userJobHelper={userJobHelper}')

        userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_NOTES)
        localDownloadPath = os.getcwd() + constants.CHAR__FORWARD_SLASH
        transcriptionsFilename = FileHelper.getFileName(userJobHelper.transcriptionsFilename, FileNameOption.Filename)
        slideTransistionsFilename = FileHelper.getFileName(userJobHelper.slideTransitionsFilename, FileNameOption.Filename)
        presentationFilename = FileHelper.getFileName(userJobHelper.presentationFilename, FileNameOption.Filename)

        createdPresentation = False
        Logger.log(f'CreatePresentation -> localDownloadPath={localDownloadPath}{constants.CHAR__NEWLINE} transcriptionsFilename={transcriptionsFilename}{constants.CHAR__NEWLINE} slideTransistionsFilename={slideTransistionsFilename}{constants.CHAR__NEWLINE} presentationFilename={presentationFilename}')
        region = boto3.Session().region_name

        #Create an instance of the SQS helper class
        userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_SENDING_EMAIL_NOTIFICATION)
        sqsHelper = SqsHelper(region, userJobHelper.appConfig.SendEmailQueueName)
        message = SqsMessage(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_SENDING_EMAIL_NOTIFICATION, {
                                    constants.UGH_PC_DDB_PK__USER_EMAIL_ID:userEmailId,
                                    constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId,
                                    constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:presentationFilename
                                })
        Logger.log(f'CreatePresentation -> Sending Message={message} to Queue={userJobHelper.appConfig.SendEmailQueueName}')
        sqsHelper.sendMessage(sqsHelper.queueName, message)
        userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_PRESENTATION, constants.UGH_PC_DDB_FLDVALUE__STATUS_COMPLETE)
        createdPresentation = True
        Logger.log(f'CreatePresentation -> {"Completed Successfully!!!" if createdPresentation else "Failed!!!"}')
    
    except Exception as e:
        Logger.log(f'CreatePresentation -> ERROR = {e}', LogLevel.Exception)       
   
    return createdPresentation

In [ ]:
sqsEvent = {
  "Records": [
    {
      "messageId": "059f36b4-87a3-44ab-83d2-661975830a7d",
      "receiptHandle": "AQEBwJnKyrHigUMZj6rYigCgxlaS3SLy0a...",
      "body": "CreatePresentation-1",
      "messageAttributes": {'user_email_id': {'DataType': 'String', 'StringValue': 'pasamoha@amazon.com'}, 'job_id': {'DataType': 'String', 'StringValue': 'baf497bb-c902-42a1-ba06-fcad41e264e'}},
      "md5OfBody": "098f6bcd4621d373cade4e832627b4f6",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-west-2:192399867703:ugh-pca_Send_Email",
      "awsRegion": "us-west-2"
    },
    {
      "messageId": "446c6e66-983e-4728-a80e-de2799ff4119",
      "receiptHandle": "AQEBwJnKyrHigUMZj6rYigCgxlaS3SLy0a...",
      "body": "CreatePresentation-2",
      "messageAttributes": {'user_email_id': {'DataType': 'String', 'StringValue': 'pasamoha@amazon.com'}, 'job_id': {'DataType': 'String', 'StringValue': 'baf497bb-c902-42a1-ba06-fcad41e264e'}},
      "md5OfBody": "098f6bcd4621d373cade4e832627b4f6",
      "eventSource": "aws:sqs",
      "eventSourceARN": "arn:aws:sqs:us-west-2:192399867703:ugh-pca_Send_Email",
      "awsRegion": "us-west-2"
    }
  ]
}

processEvent(sqsEvent)

In [ ]:
 
    
def createPresentation(userEmailId: str, jobId: str) -> bool:
    createdPresentation = False
    Logger.log(f'CreatePresentation -> User={userEmailId}{constants.CHAR__NEWLINE} jobId={jobId}')
    
    try:
        # Create New Job when Job ID exists
        userJobHelper = UserJobHelper(userEmailId, jobId)
        Logger.log(f'CreatePresentation -> userJobHelper={userJobHelper}')

        userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_NOTES)
        localDownloadPath = os.getcwd() + constants.CHAR__FORWARD_SLASH
        transcriptionsFilename=FileHelper.getFileName(userJobHelper.transcriptionsFilename, FileNameOption.Filename)
        slideTransistionsFilename = FileHelper.getFileName(userJobHelper.slideTransitionsFilename, FileNameOption.Filename)
        presentationFilename = FileHelper.getFileName(userJobHelper.presentationFilename, FileNameOption.Filename)
        createdPresentation = False
        Logger.log(f'CreatePresentation -> localDownloadPath={localDownloadPath}{constants.CHAR__NEWLINE} transcriptionsFilename={transcriptionsFilename}{constants.CHAR__NEWLINE} slideTransistionsFilename={slideTransistionsFilename}{constants.CHAR__NEWLINE} presentationFilename={presentationFilename}')
        region = boto3.Session().region_name
        s3Helper = S3Helper(region)
        continueFlag = False
        jobStatus = constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_PROCESSING_VIDEO
        
        if s3Helper.downloadFile(userJobHelper.bucketname, userJobHelper.slideTransitionsFilename, localDownloadPath + slideTransistionsFilename):
            if s3Helper.downloadFile(userJobHelper.bucketname, userJobHelper.transcriptionsFilename, localDownloadPath + transcriptionsFilename):
                slideTranscriptionsFilename = FileHelper.getFileName(userJobHelper.slideTranscriptionsFilename, FileNameOption.Filename)    
                presentationFileExtension = FileHelper.getFileName(userJobHelper.presentationTemplateFilename, FileNameOption.FilenameLastExtension)
                uploadPresentationFilename = FileHelper.getFileName(userJobHelper.videoFilename, FileNameOption.FilenamePath) + jobId + presentationFileExtension
                Logger.log(f'CreatePresentation -> slideTranscriptionsFilename={slideTranscriptionsFilename}{constants.CHAR__NEWLINE} presentationFileExtension={presentationFileExtension}{constants.CHAR__NEWLINE} uploadPresentationFilename={uploadPresentationFilename}')
                presentationHelper = PresentationHelper(slideTransistionsFilename, transcriptionsFilename)
                PresentationHelper.toJsonFile(presentationHelper.getNotes(), slideTranscriptionsFilename)
                userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_PRESENTATION)
                s3Helper.deleteFile(userJobHelper.bucketname, userJobHelper.slideTranscriptionsFilename)

                if userJobHelper.presentationFilename != constants.CHAR__EMPTY and s3Helper.downloadFile(userJobHelper.bucketname, userJobHelper.presentationFilename, localDownloadPath + presentationFilename):
                    Logger.log(f'CreatePresentation -> Downloaded Presentation Filename={userJobHelper.presentationFilename}')
                    continueFlag = True
                else:
                    slidePrefix = FileHelper.getFileName(userJobHelper.slidePrefix, FileNameOption.Filename) + constants.CHAR__FORWARD_SLASH
                    Logger.log(f'CreatePresentation -> Trying to download all slide images from slides prefix={userJobHelper.slidePrefix} to Location={localDownloadPath + slidePrefix}')
                    if s3Helper.downloadFiles(userJobHelper.bucketname, userJobHelper.slidePrefix, localDownloadPath + slidePrefix):
                        presentationFilename = jobId + presentationFileExtension
                        if s3Helper.downloadFile(userJobHelper.bucketname, userJobHelper.presentationTemplateFilename, localDownloadPath + presentationFilename):
                            Logger.log(f'CreatePresentation -> Downloaded presentationTemplateFilename={userJobHelper.presentationTemplateFilename} as Presentation Filename={userJobHelper.presentationFilename}')
                            continueFlag = True

                if continueFlag:
                    addSlideNotesToPresentationArgs = {
                            constants.UGH_PC_DDB_FLD__AC_SLIDE_TRANSCRIPTIONS_FILENAME:  slideTranscriptionsFilename,
                            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME: presentationFilename,
                            constants.UGH_PC_DDB_FLD__AC_PRESENTATION_TEMPLATE_FILENAME: presentationFilename,
                            constants.UGH_PC_DDB_FLD__AC_SLIDES_PREFIX: slidePrefix,
                            constants.UGH_PC_DDB_FLD__OVERWRITE_NOTES : userJobHelper.overwriteNotes
                        }
                    if PresentationHelper.addSlideNotesToPresentation(addSlideNotesToPresentationArgs):
                        Logger.log(f'CreatePresentation -> Presentation notes added successfully - addSlideNotesToPresentationArgs={addSlideNotesToPresentationArgs}')
                        if s3Helper.uploadFile(userJobHelper.bucketname, userJobHelper.slideTranscriptionsFilename, localDownloadPath + slideTranscriptionsFilename) and s3Helper.uploadFile(userJobHelper.bucketname, uploadPresentationFilename, localDownloadPath + presentationFilename):
                            Logger.log(f'CreatePresentation -> Uploaded slideTranscriptionsFilename={userJobHelper.slideTranscriptionsFilename} and presentation={uploadPresentationFilename}')
                            createdPresentation = True
            else:
                Logger.log(f'CreatePresentation -> transcriptionsFilename={transcriptionsFilename} not found and will retry after sometime')
                jobStatus = constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_PROCESSING_AUDIO
        else:
            Logger.log(f'CreatePresentation -> slideTransistionsFilename={slideTransistionsFilename} not found and will retry after sometime')
            
        if continueFlag == True:
            userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_PRESENTATION,  constants.UGH_PC_DDB_FLDVALUE__STATUS_COMPLETE if createdPresentation else constants.UGH_PC_DDB_FLDVALUE__STATUS_FAILED)

            #Create an instance of the SQS helper class
            userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_SENDING_EMAIL_NOTIFICATION)
            sqsHelper = SqsHelper(region, userJobHelper.appConfig.SendEmailQueueName)
            message = SqsMessage(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_SENDING_EMAIL_NOTIFICATION, {
                                        constants.UGH_PC_DDB_PK__USER_EMAIL_ID:userEmailId,
                                        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId,
                                        constants.UGH_PC_DDB_FLD__AC_PRESENTATION_FILENAME:presentationFilename
                                    })
            Logger.log(f'CreatePresentation -> Sending Message={message} to Queue={userJobHelper.appConfig.SendEmailQueueName}')
            sqsHelper.sendMessage(sqsHelper.queueName, message)
            userJobHelper.updateJobStatus(constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_PRESENTATION, constants.UGH_PC_DDB_FLDVALUE__STATUS_COMPLETE)
            Logger.log(f'CreatePresentation -> {"Completed Successfully!!!" if createdPresentation else "Failed!!!"}')
        else:
            userJobHelper.updateJobStatus(jobStatus)
            #Reinsert the message into the Queue for processing it later
            sqsHelper = SqsHelper(region, userJobHelper.appConfig.CreatePresentationQueueName)
            message = SqsMessage(constants.UGH_PC_DDB_FLDVALUE__STATUS_RETRY + constants.UGH_PC_DDB_FLDVALUE__JOB_STATUS_CREATING_NOTES, {
                                        constants.UGH_PC_DDB_PK__USER_EMAIL_ID:userEmailId,
                                        constants.UGH_PC_DDB_SK__USER_JOB_ID: jobId
                                    })
            Logger.log(f'CreatePresentation -> Retry later: Sending Message={message} to Queue={userJobHelper.appConfig.SendEmailQueueName}')
            sqsHelper.sendMessage(sqsHelper.queueName, message)
            
    
    except Exception as e:
        Logger.log(f'CreatePresentation -> ERROR = {e}', LogLevel.Exception)       
   
    return createdPresentation

In [ ]:
userEmailId = 'pasamoha@amazon.com'
jobId = 'baf497bb-c902-42a1-ba06-fcad41e264e'

createdPresentation = createPresentation(userEmailId, jobId)

In [ ]:
pip install scenedetect[opencv]

In [ ]:
from __future__ import print_function
import os

# Standard PySceneDetect imports:
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
# For caching detection metrics and saving/loading to a stats file
from scenedetect.stats_manager import StatsManager

# For content-aware scene detection:
from scenedetect.detectors.content_detector import ContentDetector


def find_scenes(video_path):
    # type: (str) -> List[Tuple[FrameTimecode, FrameTimecode]]
    video_manager = VideoManager([video_path])
    stats_manager = StatsManager()
    # Construct our SceneManager and pass it our StatsManager.
    scene_manager = SceneManager(stats_manager)

    # Add ContentDetector algorithm (each detector's constructor
    # takes detector options, e.g. threshold).
    scene_manager.add_detector(ContentDetector())

    # We save our stats file to {VIDEO_PATH}.stats.csv.
    stats_file_path = '%s.stats.csv' % video_path

    scene_list = []

    try:
        # If stats file exists, load it.
        if os.path.exists(stats_file_path):
            # Read stats from CSV file opened in read mode:
            with open(stats_file_path, 'r') as stats_file:
                stats_manager.load_from_csv(stats_file)

        # Set downscale factor to improve processing speed.
        video_manager.set_downscale_factor()

        # Start video_manager.
        video_manager.start()

        # Perform scene detection on video_manager.
        scene_manager.detect_scenes(frame_source=video_manager)

        # Obtain list of detected scenes.
        scene_list = scene_manager.get_scene_list()
        # Each scene is a tuple of (start, end) FrameTimecodes.

        print('List of scenes obtained:')
        for i, scene in enumerate(scene_list):
            print(
                'Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
                i+1,
                scene[0].get_timecode(), scene[0].get_frames(),
                scene[1].get_timecode(), scene[1].get_frames(),))

        # We only write to the stats file if a save is required:
        if stats_manager.is_save_required():
            base_timecode = video_manager.get_base_timecode()
            with open(stats_file_path, 'w') as stats_file:
                stats_manager.save_to_csv(stats_file, base_timecode)

    finally:
        video_manager.release()

    return scene_list

In [ ]:
video_path = 'my-presentation.mp4'
find_scenes(video_path)